In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import SparsePCA
import scipy
from scipy import io
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', 500)

In [47]:
df_user = pd.read_csv('yelp_data/user1.csv', usecols=[15, 16, 18, 19, 21, 22])
# df_checkin = pd.read_csv('yelp_data/business.csv', index_col=0)
# df_tip = pd.read_csv('yelp_data/tip.csv', index_col=0)
df_business = pd.read_csv('yelp_data/business.csv', usecols=[1, 3, 4, 5, 10])
df_business.rename(columns={'name':'business_name', 'city':'business_city'}, inplace=True)
df_review = pd.read_csv('yelp_data/review.csv', usecols=[1, 10])

In [3]:
df_user.head()

,fans,friends,name,review_count,useful,user_id
0,298,"[iJg9ekPzF9lkMuvjKYX6uA, ctWAuzS04Xu0lke2Rop4l...",Rob,761,18456,EZmocAborM6z66rTzeZxzQ
1,34,"[r2UUCzGxqI6WPsiWPgqG2A, qewG3X2O4X6JKskxyyqFw...",Vivian,80,117,myql3o3x22_ygECb8gVo7A
2,48,"[qewG3X2O4X6JKskxyyqFwQ, -50XWnmQGqBgEI-9ANvLl...",Carol,841,58,FIk4lQQu1eTe2EpzQ4xhBA
3,28,"[AIhfuFmX62k7a22gXXAB2Q, kQ1jU2rDpFD_q54edEmwI...",Miss Maggie,376,42,ojovtd9c8GIeDiB8e0mq2w
4,2,"[bvZkBCY-AvCsxRjd6Q-vEQ, KNEVNYeyNZApXG6HqOdsR...",Jeremy,28,8,uVEoZmmL9yK0NMgadLL0CQ


In [4]:
df_business.head()

,address,business_id,categories,city,business_name
0,"227 E Baseline Rd, Ste J2",0DI8Dt2PJp07XkVvIElIcQ,"[Tobacco Shops, Nightlife, Vape Shops, Shopping]",Tempe,Innovative Vapors
1,495 S Grand Central Pkwy,LTlCaCGZE14GuaUXUGbamg,"[Caterers, Grocery, Food, Event Planning & Ser...",Las Vegas,Cut and Taste
2,979 Bloor Street W,EDqCEAGXVGCH4FJXgqtjqg,"[Restaurants, Pizza, Chicken Wings, Italian]",Toronto,Pizza Pizza
3,7014 Steubenville Pike,cnGIivYRLxpF7tBVR_JwWA,"[Hair Removal, Beauty & Spas, Blow Dry/Out Ser...",Oakdale,Plush Salon and Spa
4,321 Jarvis Street,cdk-qqJ71q6P7TJTww_DSA,"[Hotels & Travel, Event Planning & Services, H...",Toronto,Comfort Inn


In [5]:
df_review.head()

,business_id,user_id
0,2aFiy99vNLklCx3T_tGS9A,KpkOkG6RIf4Ra25Lhhxf1A
1,2aFiy99vNLklCx3T_tGS9A,bQ7fQq1otn9hKX-gXRsrgA
2,2aFiy99vNLklCx3T_tGS9A,r1NUhdNmL6yU9Bn-Yx6FTw
3,2LfIuF3_sX6uwe-IR-P0jQ,aW3ix1KNZAvoM8q-WghA3Q
4,2LfIuF3_sX6uwe-IR-P0jQ,YOo-Cip8HqvKp_p9nEGphw


In [6]:
df = pd.merge(pd.merge(df_user, df_review, how='inner', on='user_id'), df_business, how='inner', on='business_id')
df = df.drop(np.array(df[df.friends == '[None]'].index), axis=0)
df = df.dropna()[:2000000]

In [7]:
df1 = df.drop_duplicates('business_id')

In [8]:
pattern = '|'.join(['restaurant'])
df1 = df1[df1.categories.str.lower().str.replace('[', '').str.replace(']', '').str.replace(',', '|').str.contains(pattern)]


In [9]:
unique_words = set()
for wordlist in df1.categories.str.replace('[', '').str.replace(']', '').str.split(',').values:
    if wordlist is not np.nan:
        unique_words = unique_words.union(set(wordlist))
categories_wordbag = list(unique_words)
for word in categories_wordbag:
    df1['categories_' + word.replace(' ', '_')] = df1.categories.str.contains(word).astype(float)


/Users/zengfanchen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [10]:
df1 = df1.drop(['categories', 'fans', 'friends', 'name', 'review_count', 'useful', 'city', 'user_id'], axis=1)
df = df.drop('categories', axis=1)

In [11]:
df = pd.merge(df, df1, how='inner', on='business_id', sort=True).drop(['categories__Restaurants', 'categories_Canadian_(New)', 'categories__American_(Traditional)'], axis=1)

In [12]:
X = df.drop(['fans', 'friends', 'name', 'review_count', 'useful', 'business_id'], axis=1)
X.head()

,user_id,address_x,city,business_name_x,address_y,business_name_y,categories__Auto_Repair,categories__Iberian,categories_Drugstores,categories__Falafel,categories_Buffets,categories_Curry_Sausage,categories__Health_Markets,categories_Creperies,categories__Cosmetics_&_Beauty_Supply,categories_Teppanyaki,categories_Pretzels,categories__Gelato,categories__Party_&_Event_Planning,categories__Country_Dance_Halls,categories__Breakfast_&_Brunch,categories__Zoos,categories__Pan_Asian,categories_Juice_Bars_&_Smoothies,categories__Vacation_Rentals,categories_Grocery,categories__Cooking_Schools,categories__Golf,categories__Nail_Salons,categories__Sports_Clubs,categories_Country_Dance_Halls,categories__Peruvian,categories__Sports_Bars,categories_Bowling,categories__Cafeteria,categories__Local_Flavor,categories_Restaurants,categories__Poke,categories__Ethnic_Food,categories_Playgrounds,categories_Delis,categories_Halal,categories_Cantonese,categories_Herbs_&_Spices,categories__Cocktail_Bars,categories_Noodles,categories__Barbeque,categories__Gas_&_Service_Stations,categories_Chicken_Wings,categories__Shopping,categories__Hotels_&_Travel,categories_Antiques,categories_Food_Stands,categories_Shopping_Centers,categories__Lounges,categories__Furniture_Stores,categories__British,categories_Automotive,categories__Social_Clubs,categories__Italian,categories__Food_Delivery_Services,categories__Wineries,categories_Lounges,categories_Persian/Iranian,categories__Building_Supplies,categories__Sushi_Bars,categories_Cafes,categories_Cajun/Creole,categories__Dance_Clubs,categories_Scandinavian,categories_Greek,categories_Zoos,categories__Hotels,categories_Chinese,categories__Bangladeshi,categories__Chicken_Shop,categories_Food_Trucks,categories_Health_&_Medical,categories_Organic_Stores,categories__Boating,categories__Car_Dealers,categories__Mortgage_Brokers,categories__Beer_Gardens,categories__Hawaiian,categories__Tobacco_Shops,categories__Mags,categories_Hookah_Bars,categories_Middle_Eastern,categories_Macarons,categories__Health_&_Medical,categories__Horseback_Riding,categories__Pediatricians,categories__Specialty_Food,categories__Ramen,categories_Shanghainese,categories__Hostels,categories_Hungarian,categories__Uzbek,categories_Escape_Games,categories_Souvenir_Shops,categories__Kosher,categories__Food_Stands,categories_Diners,categories_Sports_Bars,categories_Russian,categories_Seafood_Markets,categories__Seafood_Markets,categories__German,categories_Vegetarian,categories__Ethiopian,categories_Kosher,categories__Cabaret,categories_Social_Clubs,categories__Halal,categories__Reunion,categories__Hot_Pot,categories__Drugstores,categories_German,categories_Taiwanese,categories_Laotian,categories__Cardiologists,categories_Basque,categories__Pizza,categories__Imported_Food,categories__South_African,categories__Chocolatiers_&_Shops,categories_Vietnamese,categories__Pubs,categories__Szechuan,categories_Australian,categories_Venezuelan,categories_Pizza,categories__Comfort_Food,categories__Vegetarian,categories__Flea_Markets,categories__Adult_Entertainment,categories_Food_Court,categories__Food_Court,categories_Music_Venues,categories__Piano_Bars,categories_RV_Parks,categories_Seafood,categories__Czech,categories_British,categories__Mediterranean,categories__Pasta_Shops,categories__Waffles,categories_South_African,categories_Beer_Gardens,categories__Yelp_Events,categories__Dinner_Theater,categories_Japanese,categories__Leisure_Centers,categories__Brazilian,categories_Hot_Dogs,categories_Gift_Shops,categories__Wedding_Planning,categories_Nicaraguan,categories__Beer_Bar,categories__Nutritionists,categories__Candy_Stores,categories_Salvadoran,categories__Slovakian,categories_Sandwiches,categories__International_Grocery,categories_Shaved_Ice,categories__Venues_&_Event_Spaces,categories__Irish_Pub,categories_Dim_Sum,categories__Bistros,categories__Patisserie/Cake_Shop,categories__Shopping_Centers,categories__Bowling,categories__Do-It-Yourself_Food,categories__Museums

In [13]:
X_city = X[['user_id','city']]
X_city.head()

,user_id,city
0,5gbkhlWANGVzQYi_0m3vOQ,Las Vegas
1,CjvFxl6_7AK3lw5O49MN8g,Las Vegas
2,R0KVWeN9xR-F6j4z5k9wXQ,Las Vegas
3,_Dd2uxnhcEKzrv0VPVnxFA,Las Vegas
4,qLMT6tS074Opsaz4qkbjVg,Las Vegas


In [14]:
grouped = X.groupby('user_id', as_index=0).sum()

In [15]:
grouped_city = pd.merge(grouped, X_city, how='inner', on='user_id')
grouped = grouped_city.drop_duplicates('user_id')

In [16]:
cor = grouped.drop('user_id', axis=1).corr()
scor = scipy.sparse.csr_matrix(cor.values)
cor_criteria = 0.8
p = []
for i in xrange(627):
    for j in xrange(i+1, 627):
        if scor[i, j] > cor_criteria:
            p.append(j+1)
p = list(set(p))
grouped = grouped.drop(grouped.columns[p], axis=1)


In [17]:
del_index = []
for i in xrange(len(grouped)):
    if np.count_nonzero(grouped.iloc[[i]].values[0][1:]) == 0:
        del_index.append(i)
grouped = grouped.drop(grouped.index[p], axis=0)  

In [18]:
grouped_city = grouped.drop_duplicates('user_id')
grouped_city.to_csv('user_detail.csv', sep=',', encoding='utf-8')

In [19]:
grouped_city.head()

,user_id,categories__Auto_Repair,categories__Iberian,categories_Drugstores,categories__Falafel,categories_Buffets,categories_Curry_Sausage,categories__Health_Markets,categories_Creperies,categories__Cosmetics_&_Beauty_Supply,categories_Teppanyaki,categories_Pretzels,categories__Gelato,categories__Party_&_Event_Planning,categories__Country_Dance_Halls,categories__Breakfast_&_Brunch,categories__Zoos,categories__Pan_Asian,categories_Juice_Bars_&_Smoothies,categories__Vacation_Rentals,categories_Grocery,categories__Cooking_Schools,categories__Golf,categories__Nail_Salons,categories__Sports_Clubs,categories__Peruvian,categories__Sports_Bars,categories_Bowling,categories__Cafeteria,categories__Local_Flavor,categories__Poke,categories__Ethnic_Food,categories_Playgrounds,categories_Delis,categories_Halal,categories_Cantonese,categories_Herbs_&_Spices,categories__Cocktail_Bars,categories_Noodles,categories__Barbeque,categories__Gas_&_Service_Stations,categories_Chicken_Wings,categories__Shopping,categories__Hotels_&_Travel,categories_Antiques,categories_Food_Stands,categories_Shopping_Centers,categories__Lounges,categories__Furniture_Stores,categories__British,categories_Automotive,categories__Social_Clubs,categories__Italian,categories__Food_Delivery_Services,categories__Wineries,categories_Persian/Iranian,categories__Building_Supplies,categories__Sushi_Bars,categories_Cafes,categories_Cajun/Creole,categories__Dance_Clubs,categories_Scandinavian,categories_Greek,categories_Chinese,categories__Bangladeshi,categories__Chicken_Shop,categories_Food_Trucks,categories_Health_&_Medical,categories_Organic_Stores,categories__Boating,categories__Mortgage_Brokers,categories__Beer_Gardens,categories__Hawaiian,categories__Tobacco_Shops,categories__Mags,categories_Hookah_Bars,categories_Middle_Eastern,categories_Macarons,categories__Pediatricians,categories__Specialty_Food,categories__Ramen,categories_Shanghainese,categories__Hostels,categories_Hungarian,categories__Uzbek,categories_Escape_Games,categories_Souvenir_Shops,categories__Kosher,categories_Diners,categories_Russian,categories_Seafood_Markets,categories__German,categories_Vegetarian,categories__Ethiopian,categories__Cabaret,categories__Reunion,categories__Hot_Pot,categories_German,categories_Taiwanese,categories_Laotian,categories_Basque,categories__Pizza,categories__Imported_Food,categories__South_African,categories__Chocolatiers_&_Shops,categories_Vietnamese,categories__Pubs,categories__Szechuan,categories_Australian,categories_Venezuelan,categories__Comfort_Food,categories__Flea_Markets,categories__Adult_Entertainment,categories_Food_Court,categories_Music_Venues,categories__Piano_Bars,categories_RV_Parks,categories_Seafood,categories__Czech,categories__Mediterranean,categories__Pasta_Shops,categories__Waffles,categories_South_African,categories__Yelp_Events,categories__Dinner_Theater,categories_Japanese,categories__Brazilian,categories_Hot_Dogs,categories_Gift_Shops,categories__Wedding_Planning,categories_Nicaraguan,categories__Beer_Bar,categories__Nutritionists,categories__Candy_Stores,categories_Salvadoran,categories__Slovakian,categories__International_Grocery,categories_Shaved_Ice,categories__Venues_&_Event_Spaces,categories__Irish_Pub,categories_Dim_Sum,categories__Bistros,categories__Patisserie/Cake_Shop,categories__Do-It-Yourself_Food,categories__Museums,categories__Pet_Stores,categories__Couriers_&_Delivery_Services,categories__Mexican,categories__Fashion,categories__Vegan,categories__Public_Services_&_Government,categories__Scottish,categories__Fish_&_Chips,categories_Dive_Bars,categories__Fondue,categories_Moroccan,categories_Active_Life,categories__Ethnic_Grocery,categories_Latin_American,categories__Automotive,categories_Beauty_&_Spas,categories__Meat_Shops,categories__Arts_&_Crafts,categories__Keys_&_Locksmiths,categories__Puerto_Rican,categories__Outlet_Stores,categories__Pumpkin_Patches,categories_Hair_Salons,categories_Coffee_&_Tea,categories_Cheesesteaks,categories__F

In [ ]:
# cor_user = grouped.drop('user_id', axis=1).T.corr()
# scor_user = scipy.sparse.csr_matrix(cor_user.values)
# scor_user.shape

In [ ]:
# p[:] = [x - 1 for x in p]
# cor_matrix = scor.toarray()
# cor_matrix = np.delete(cor_matrix, p, axis=0)
# cor_matrix = np.delete(cor_matrix, p, axis=1)
# cor_matrix[np.diag_indices_from(cor_matrix)] = -1.
# print cor_matrix[0:].argmax()

In [20]:
sca = MinMaxScaler()
grouped_sca = pd.DataFrame(sca.fit_transform(grouped_city.drop(['user_id', 'city'], axis=1)), 
                           index=grouped_city.index.values, 
                           columns=grouped_city.columns.values[1:-1])
x = grouped_sca
x.head()

,categories__Auto_Repair,categories__Iberian,categories_Drugstores,categories__Falafel,categories_Buffets,categories_Curry_Sausage,categories__Health_Markets,categories_Creperies,categories__Cosmetics_&_Beauty_Supply,categories_Teppanyaki,categories_Pretzels,categories__Gelato,categories__Party_&_Event_Planning,categories__Country_Dance_Halls,categories__Breakfast_&_Brunch,categories__Zoos,categories__Pan_Asian,categories_Juice_Bars_&_Smoothies,categories__Vacation_Rentals,categories_Grocery,categories__Cooking_Schools,categories__Golf,categories__Nail_Salons,categories__Sports_Clubs,categories__Peruvian,categories__Sports_Bars,categories_Bowling,categories__Cafeteria,categories__Local_Flavor,categories__Poke,categories__Ethnic_Food,categories_Playgrounds,categories_Delis,categories_Halal,categories_Cantonese,categories_Herbs_&_Spices,categories__Cocktail_Bars,categories_Noodles,categories__Barbeque,categories__Gas_&_Service_Stations,categories_Chicken_Wings,categories__Shopping,categories__Hotels_&_Travel,categories_Antiques,categories_Food_Stands,categories_Shopping_Centers,categories__Lounges,categories__Furniture_Stores,categories__British,categories_Automotive,categories__Social_Clubs,categories__Italian,categories__Food_Delivery_Services,categories__Wineries,categories_Persian/Iranian,categories__Building_Supplies,categories__Sushi_Bars,categories_Cafes,categories_Cajun/Creole,categories__Dance_Clubs,categories_Scandinavian,categories_Greek,categories_Chinese,categories__Bangladeshi,categories__Chicken_Shop,categories_Food_Trucks,categories_Health_&_Medical,categories_Organic_Stores,categories__Boating,categories__Mortgage_Brokers,categories__Beer_Gardens,categories__Hawaiian,categories__Tobacco_Shops,categories__Mags,categories_Hookah_Bars,categories_Middle_Eastern,categories_Macarons,categories__Pediatricians,categories__Specialty_Food,categories__Ramen,categories_Shanghainese,categories__Hostels,categories_Hungarian,categories__Uzbek,categories_Escape_Games,categories_Souvenir_Shops,categories__Kosher,categories_Diners,categories_Russian,categories_Seafood_Markets,categories__German,categories_Vegetarian,categories__Ethiopian,categories__Cabaret,categories__Reunion,categories__Hot_Pot,categories_German,categories_Taiwanese,categories_Laotian,categories_Basque,categories__Pizza,categories__Imported_Food,categories__South_African,categories__Chocolatiers_&_Shops,categories_Vietnamese,categories__Pubs,categories__Szechuan,categories_Australian,categories_Venezuelan,categories__Comfort_Food,categories__Flea_Markets,categories__Adult_Entertainment,categories_Food_Court,categories_Music_Venues,categories__Piano_Bars,categories_RV_Parks,categories_Seafood,categories__Czech,categories__Mediterranean,categories__Pasta_Shops,categories__Waffles,categories_South_African,categories__Yelp_Events,categories__Dinner_Theater,categories_Japanese,categories__Brazilian,categories_Hot_Dogs,categories_Gift_Shops,categories__Wedding_Planning,categories_Nicaraguan,categories__Beer_Bar,categories__Nutritionists,categories__Candy_Stores,categories_Salvadoran,categories__Slovakian,categories__International_Grocery,categories_Shaved_Ice,categories__Venues_&_Event_Spaces,categories__Irish_Pub,categories_Dim_Sum,categories__Bistros,categories__Patisserie/Cake_Shop,categories__Do-It-Yourself_Food,categories__Museums,categories__Pet_Stores,categories__Couriers_&_Delivery_Services,categories__Mexican,categories__Fashion,categories__Vegan,categories__Public_Services_&_Government,categories__Scottish,categories__Fish_&_Chips,categories_Dive_Bars,categories__Fondue,categories_Moroccan,categories_Active_Life,categories__Ethnic_Grocery,categories_Latin_American,categories__Automotive,categories_Beauty_&_Spas,categories__Meat_Shops,categories__Arts_&_Crafts,categories__Keys_&_Locksmiths,categories__Puerto_Rican,categories__Outlet_Stores,categories__Pumpkin_Patches,categories_Hair_Salons,categories_Coffee_&_Tea,categories_Cheesesteaks,categories__Fruits_&_

In [21]:
M = x.div(x.sum(axis=1), axis=0).fillna(value=0.)
G = x/x.sum().astype(np.float64).fillna(value=0.)
M = scipy.sparse.csr_matrix(M.values)
G = scipy.sparse.csr_matrix(G.values).T
m = 1
GM = G.dot(M)**m
pi = GM.dot(x.values[0,:])
pi[np.isnan(pi)] = 0.


In [22]:
io.mmwrite('diffsion operator',GM )


In [50]:
df_business.head()

,address,business_id,categories,business_city,business_name,categories__Iberian,categories_Masonry/Concrete,categories_Drugstores,categories_Dry_Cleaning_&_Laundry,categories__Falafel,categories_Buffets,categories__Armenian,categories_Curry_Sausage,categories__Health_Markets,categories_Creperies,categories_Canadian_(New),categories__Septic_Services,categories__Cosmetics_&_Beauty_Supply,categories_Teppanyaki,categories_Pretzels,categories_Fishmonger,categories_Organic_Stores,categories_Salad,categories__Auto_Insurance,categories__Party_&_Event_Planning,categories__Country_Dance_Halls,categories__Breakfast_&_Brunch,categories__Zoos,categories__Pan_Asian,categories__Restaurants,categories_Juice_Bars_&_Smoothies,categories__Vacation_Rentals,categories_Grocery,categories__Cooking_Schools,categories__Golf,categories__Hair_Salons,categories__Sports_Clubs,categories_Country_Dance_Halls,categories__Peruvian,categories_Contractors,categories__Sports_Bars,categories__Interior_Design,categories__Massage_Therapy,categories_Bowling,categories__Cafeteria,categories__Local_Flavor,categories_Restaurants,categories__Poke,categories__Ethnic_Food,categories__Dance_Schools,categories_Playgrounds,categories__Hair_Removal,categories__Parks,categories__Trainers,categories_Delis,categories_Bed_&_Breakfast,categories_Halal,categories_Cantonese,categories_Herbs_&_Spices,categories__Cocktail_Bars,categories_Noodles,categories__Barbeque,categories__Acupuncture,categories__Speakeasies,categories_Chicken_Wings,categories__DJs,categories__Irish,categories__RV_Repair,categories_Antiques,categories__Wedding_Chapels,categories_Kitchen_&_Bath,categories_Food_Stands,categories_Ethiopian,categories_Shopping_Centers,categories__Lounges,categories__Furniture_Stores,categories__Masonry/Concrete,categories__British,categories__Gardeners,categories__Limos,categories__Pool_&_Hot_Tub_Service,categories_Elementary_Schools,categories_Automotive,categories__Golf_Lessons,categories__CSA,categories__Social_Clubs,categories__Italian,categories__Food_Delivery_Services,categories__Wineries,categories_Lounges,categories_Persian/Iranian,categories__Building_Supplies,categories__Sushi_Bars,categories_Cajun/Creole,categories__Dance_Clubs,categories_Scandinavian,categories_Greek,categories_Zoos,categories__Religious_Organizations,categories_Art_Galleries,categories__Hotels,categories__Skin_Care,categories_Chinese,categories__Bangladeshi,categories_Home_Decor,categories__Chicken_Shop,categories_Food_Trucks,categories_Resorts,categories__Oriental,categories__Gelato,categories__Bridal,categories__Boating,categories_Alsatian,categories__Car_Dealers,categories__Mortgage_Brokers,categories__Hainan,categories__Beer_Gardens,categories__Beach_Bars,categories__Hawaiian,categories__Tobacco_Shops,categories__Mags,categories_Hookah_Bars,categories__Southern,categories_Macarons,categories__Health_&_Medical,categories__Horseback_Riding,categories__Landscaping,categories__Specialty_Food,categories__Ramen,categories_Shanghainese,categories_Restaurant_Supplies,categories__Rotisserie_Chicken,categories_Hungarian,categories__Kids_Activities,categories__Uzbek,categories_Escape_Games,categories__Advertising,categories_Souvenir_Shops,categories__Kosher,categories__Food_Stands,categories_Diners,categories_Sports_Bars,categories_Russian,categories_Auto_Parts_&_Supplies,categories_Seafood_Markets,categories__Jazz_&_Blues,categories_Feng_Shui,categories__Seafood_Markets,categories__German,categories_Vegetarian,categories__Street_Vendors,categories__Souvenir_Shops,categories_Kosher,categories_Flea_Markets,categories__Cabaret,categories_Social_Clubs,categories__Halal,categories__Reunion,categories__Hot_Pot,categories__General_Dentistry,categories_Couriers_&_Delivery_Services,categories__Drugstores,categories_German,categories__Appliances_&_Repair,categories__Hardware_Stores,categories_Tobacco_Shops,categories_Taiwanese,categories_Haitian,categories__Ethnic_Grocery,categories__Cardiologists,categories_Appliances,

In [52]:
pattern = '|'.join(['restaurant'])
df_business = df_business.dropna(axis=0)
df_business = df_business[df_business.categories.str.lower().str.replace('[', '').str.replace(']', '').str.replace(',', '|').str.contains(pattern)]

unique_words = set()
for wordlist in df_business.categories.str.replace('[', '').str.replace(']', '').str.split(',').values:
    if wordlist is not np.nan:
        unique_words = unique_words.union(set(wordlist))
categories_wordbag = list(unique_words)
for word in categories_wordbag:
    df_business['categories_' + word.replace(' ', '_')] = df_business.categories.str.contains(word).astype(float)
df_business = df_business.drop(['categories', 'city'], axis=1)
df_business = df_business[np.insert(grouped_city.columns.values[1:-1], -1, ['address', 'business_id', 'business_city', 'business_name'])]
df_business.to_csv('business_detail.csv', sep=',', encoding='utf-8')

In [ ]:
# cor_matrix.dump('cor_matrix') 
io.mmwrite('diffsion operator', GM)

In [ ]:
GM = io.mmread('diffsion operator.mtx')

In [ ]:
GM

In [ ]:
number = 22
habit_index = grouped_city.iloc[[number]].drop(['user_id', 'city'], axis=1).values[0].argsort()[-3:][::-1]

habit = grouped_city.columns.values[habit_index]

In [ ]:
grouped_city.iloc[[number]].drop(['user_id', 'city'], axis=1).values[0].shape

In [70]:
grouped = pd.read_csv('user_detail.csv',index_col=None)
grouped = grouped.drop(grouped.columns[0], axis=1)

In [4]:
import numpy as np
a = np.array(['abcde', 'bdcss'])
print a[0]

abcde


In [1]:
tt = X[:10]

NameError: name 'X' is not defined